## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
import random
from pprint import pprint
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from IPython.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract_bash, query_contract, get_ipfs_cid_from_str, get_proofs, ContractUtils


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 100_000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 50_000
NUMBER_OF_THREADS = 100
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [30_000] * NUMBER_OF_PARTICIPANTS
PREFIXES = ['cosmos', 'osmo', 'terra']

INITIAL_BALANCE = str(18_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(20_000)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
PYTHON_PATH = dotenv_values('.env')['PYTHON_PATH']

DISPLAY_TX_EXECUTION = False
CALCULATE_ADDRESS_SET = True
CALCULATE_PROOFS = True
SEND_COINS = True

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(3)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1qwlgtx52gsdu7dtp0cekka5zehdl0uj3fhp9acg325fvgs8jdzksa7wx60'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(2)
PASSPORT_CONTRACT_ADDRESS = 'bostrom18v47nqmhvejx3vc498pantg8vr435xa0rt6x0m6kzhp6yuqmcp8syatwkd'

INIT_TREASURY_CONTRACT = True
TREASURY_CODE_ID = str(4)
TREASURY_CONTRACT_ADDRESS = 'bostrom1wkwy0xh89ksdgj9hr347dyd2dw7zesmtrue6kfzyml4vdtz6e5ws8fj6nu'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(1)
GIFT_CONTRACT_ADDRESS = 'bostrom14swjuycwxmvghd9hgr3f3755388g6xkaqpn2dxcf700fzcwdm2nqum7453'

PARTICIPANTS_FILE_NAME = 'participants_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv'
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/home/grovybear/.local/lib/python3.9/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.12.2' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
def generate_wallet_data(i: int, prefixes: list = PREFIXES) -> dict:

    prefix = random.choice(prefixes)

    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
              'nickname': NICKNAME_LIST[i],
              'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], prefix)
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix=prefix)
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    return claim_item


if CALCULATE_ADDRESS_SET:

    tasks = list(range(NUMBER_OF_PARTICIPANTS))
    print(f'Number of tasks: {len(tasks)}')
    print(f'Number of threads: {NUMBER_OF_THREADS}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        claims_list = list(tqdm(pool.imap(generate_wallet_data, tasks), total=len(tasks)))

    claims_df = pd.DataFrame(claims_list)
    claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
else:
    claims_df = pd.read_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df.head()

Number of tasks: 100000
Number of threads: 100


  0%|          | 0/100000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,30000,john16514110440,QmVgtMwsrumhGovy8fz7eahPgdk4ehCXTR82tTufLQNeE7,0x3756069ce10d18b5d23db3295ec59f62909b4032,0xabeaa8b4a26ca6c28acc61979b5516f58cca5a93c2f3...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4,cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj,domain jacket harsh toast list evil champion s...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4...,0x9f38deff0690e29df1fe865ec1ac5af4d20b764fbde9...,{'pub_key': 'A1w2Dz/7uX7iNzNjygPpiT+/vOl7orPY4...,eyJwdWJfa2V5IjoiQTF3MkR6Lzd1WDdpTnpOanlnUHBpVC...
1,30000,john16514110441,QmVWURubuV33SKYXtyLtT4MtN1pfPrukzT9tjR4vmBRVBd,0xc70b2e789b42a6b3add29f8949b4ec8ebeb78aa9,0xbf851509658dd955365c2b62848a55842cd28937b792...,bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l,terra1mvuczu56l9c3rsj889ysykumpg4s9v3xd6wfxc,net coin wealth creek lumber reduce catalog sl...,bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l...,0x97b409a700e4d11d7b5cadd764c67cc6399b868d93e2...,{'pub_key': 'A3nBxjlMaPLs6FP7eSIMG9ztk7w/EJABw...,eyJwdWJfa2V5IjoiQTNuQnhqbE1hUExzNkZQN2VTSU1HOX...
2,30000,john16514110442,QmXudghn9jWWjzzgizpTtzZ1kJLeCK7bdgRe5ANgS9dsLt,0x22d6a3a23a348a5055caf5e9459b6b81a849c3d9,0x989e2221d4dc219182aeb5d5f9ff124406759747db73...,bostrom1yyx6wvgvr3deg70dvgsfwulgzurnz8kcwsna8a,cosmos1yyx6wvgvr3deg70dvgsfwulgzurnz8kcdr8we6,shock pipe begin detect economy finger great u...,bostrom1yyx6wvgvr3deg70dvgsfwulgzurnz8kcwsna8a...,0xcad3e647570e7f168aee49bbda543824c6dc6f47a06e...,{'pub_key': 'A1VGj0IPB2PAI0ZW25CScr//U/jJkxlDA...,eyJwdWJfa2V5IjoiQTFWR2owSVBCMlBBSTBaVzI1Q1Njci...
3,30000,john16514110443,QmUCtjvizRdrdoF6e5VQRN5pgUB4D1wafwFzTsTjt2zPoY,0xa88f6812e62e6dbae520c8cc9bf2fbe68418f7cf,0xcd8696cdafd3388c6ca8a8062d3cf038dfca1129d559...,bostrom1yhrmy9l027gsa342wwxmua0pfel2c24egm037y,cosmos1yhrmy9l027gsa342wwxmua0pfel2c24etgmzqr,air video reflect suggest own skate spray mapl...,bostrom1yhrmy9l027gsa342wwxmua0pfel2c24egm037y...,0x58b5e9de07036ab4e5f2f4b185d3f0a684aca66e4678...,{'pub_key': 'AlWQq+xr7W2FW77LivQdpCQ/G8zmRucoE...,eyJwdWJfa2V5IjoiQWxXUXEreHI3VzJGVzc3TGl2UWRwQ1...
4,30000,john16514110444,QmddRuhgqujtgojjegXtZ37QA1KgH8PhqCmVPJxSfVSfPX,0x923fda130f266134410a244b826d72433822777d,0x83c14ac2ea6dd1fccec8ef74092bc9d20d1f018c115a...,bostrom178hq4ur7ywrvlsd4lmgaldp7d27dqj5h3u7fmz,osmo178hq4ur7ywrvlsd4lmgaldp7d27dqj5h65e2nh,chase melt spread boss outdoor convince hundre...,bostrom178hq4ur7ywrvlsd4lmgaldp7d27dqj5h3u7fmz...,0xfacbb78d8c1be2246538fa7832c98eee6113ecd17655...,{'pub_key': 'AhVQjlyo7QhMi20ije0+S84RykSj76TG3...,eyJwdWJfa2V5IjoiQWhWUWpseW83UWhNaTIwaWplMCtTOD...


### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item[0],
                     'amount': str(_item[1])} for _item in claims_df[['ethereum_address', 'amount']].values.tolist()]+ \
                    [{'address': _item[0],
                      'amount': str(_item[1])} for _item in claims_df[['cosmos_address', 'amount']].values.tolist()]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
NUMBER_OF_THREADS = 15
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(2 * NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
if CALCULATE_PROOFS:
    print(f'Number of addresses: {2 * NUMBER_OF_PARTICIPANTS}\nNumber of threads: {NUMBER_OF_THREADS}\nBash size: {BASH_SIZE}\nNumber of tasks: {tasks}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tqdm(tasks, total=len(tasks)))
    assert res == [True] * len(res)

Number of addresses: 200000
Number of threads: 15
Bash size: 1000
Number of tasks: {('root_testing_source_100000_addresses.json', 'temp/proofs_13.json', 13000, 14001), ('root_testing_source_100000_addresses.json', 'temp/proofs_24.json', 24000, 25001), ('root_testing_source_100000_addresses.json', 'temp/proofs_101.json', 101000, 102001), ('root_testing_source_100000_addresses.json', 'temp/proofs_191.json', 191000, 192001), ('root_testing_source_100000_addresses.json', 'temp/proofs_23.json', 23000, 24001), ('root_testing_source_100000_addresses.json', 'temp/proofs_103.json', 103000, 104001), ('root_testing_source_100000_addresses.json', 'temp/proofs_162.json', 162000, 163001), ('root_testing_source_100000_addresses.json', 'temp/proofs_42.json', 42000, 43001), ('root_testing_source_100000_addresses.json', 'temp/proofs_189.json', 189000, 190001), ('root_testing_source_100000_addresses.json', 'temp/proofs_67.json', 67000, 68001), ('root_testing_source_100000_addresses.json', 'temp/proofs_11

  0%|          | 0/200 [00:00<?, ?it/s]

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_12.json --start_index 12000 --end_index 13001
Merkle root: 41aaa12edbc734004495fd8995040aa2526f4cd0c40b58d7304e5f12258a767c
Number of addresses in the Merkle tree: 1001
Done in 50.90s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_65.json --start_index 65000 --end_index 66001
Merkle root: 41aaa12edbc734004495fd8995040aa2526f4cd0c40b58d7304e5f12258a767c
Number of addresses in the Merkle tree: 1001
Done in 51.00s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_117.json --start_index 117000 --end_index 118001
Merkle root: 41aaa12edbc734004495fd8995040aa2526f4cd0c40b58d7304e5f12258a767c
Number of addresses in the Merkle tree: 1001
Done in 51.06s.

yarn run v1.22.18
$ ts-node index.ts --input root_testing_source_100000_addresses.json --output temp/proofs_13.json

In [5]:
if CALCULATE_PROOFS:
    roots = []
    proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
    for task in tasks:
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        roots.append(root_and_proof_json['merkle_root'])
        proofs_df = pd.concat([proofs_df, pd.DataFrame(root_and_proof_json['proofs'])], ignore_index=True)
    assert roots == [roots[0]] * len(roots)
    root = roots[0]

    cosmos_proofs_df = proofs_df[~proofs_df.address.str.startswith('0x')][['address', 'proof']]
    ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

    claims_with_proofs_df = claims_df\
        .merge(
            ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
            how='left',
            on='ethereum_address')\
        .merge(
            cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
            how='left',
            on='cosmos_address')

    claims_with_proofs_df.to_csv(PARTICIPANTS_FILE_NAME)
else:
    claims_with_proofs_df = pd.read_csv(PARTICIPANTS_FILE_NAME)
    claims_with_proofs_df.loc[:, 'ethereum_proof'] = claims_with_proofs_df['ethereum_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    claims_with_proofs_df.loc[:, 'cosmos_proof'] = claims_with_proofs_df['cosmos_proof'].map(lambda x: x.replace('\'', '').replace('[', '').replace(']', '').split(', '))
    with open('temp/proofs_1.json', 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    root = root_and_proof_json['merkle_root']
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,30000,john16514110440,QmVgtMwsrumhGovy8fz7eahPgdk4ehCXTR82tTufLQNeE7,0x3756069ce10d18b5d23db3295ec59f62909b4032,0xabeaa8b4a26ca6c28acc61979b5516f58cca5a93c2f3...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4,cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj,domain jacket harsh toast list evil champion s...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4...,0x9f38deff0690e29df1fe865ec1ac5af4d20b764fbde9...,{'pub_key': 'A1w2Dz/7uX7iNzNjygPpiT+/vOl7orPY4...,eyJwdWJfa2V5IjoiQTF3MkR6Lzd1WDdpTnpOanlnUHBpVC...,[aa1a98efd431fdd62c8065eb5a9c514ce162b6281caf7...,[423ba75dd8f55bafd9b29f4cd232ccbb4828c5904750c...
1,30000,john16514110440,QmVgtMwsrumhGovy8fz7eahPgdk4ehCXTR82tTufLQNeE7,0x3756069ce10d18b5d23db3295ec59f62909b4032,0xabeaa8b4a26ca6c28acc61979b5516f58cca5a93c2f3...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4,cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj,domain jacket harsh toast list evil champion s...,bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4...,0x9f38deff0690e29df1fe865ec1ac5af4d20b764fbde9...,{'pub_key': 'A1w2Dz/7uX7iNzNjygPpiT+/vOl7orPY4...,eyJwdWJfa2V5IjoiQTF3MkR6Lzd1WDdpTnpOanlnUHBpVC...,[aa1a98efd431fdd62c8065eb5a9c514ce162b6281caf7...,[423ba75dd8f55bafd9b29f4cd232ccbb4828c5904750c...
2,30000,john16514110441,QmVWURubuV33SKYXtyLtT4MtN1pfPrukzT9tjR4vmBRVBd,0xc70b2e789b42a6b3add29f8949b4ec8ebeb78aa9,0xbf851509658dd955365c2b62848a55842cd28937b792...,bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l,terra1mvuczu56l9c3rsj889ysykumpg4s9v3xd6wfxc,net coin wealth creek lumber reduce catalog sl...,bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l...,0x97b409a700e4d11d7b5cadd764c67cc6399b868d93e2...,{'pub_key': 'A3nBxjlMaPLs6FP7eSIMG9ztk7w/EJABw...,eyJwdWJfa2V5IjoiQTNuQnhqbE1hUExzNkZQN2VTSU1HOX...,[e6510615b06bed677416e4eaf2766262a0c0ea6b5f69a...,[e7a25331b0a1a98a5bb2d516932e6ca104cc0e8d22643...
3,30000,john16514110442,QmXudghn9jWWjzzgizpTtzZ1kJLeCK7bdgRe5ANgS9dsLt,0x22d6a3a23a348a5055caf5e9459b6b81a849c3d9,0x989e2221d4dc219182aeb5d5f9ff124406759747db73...,bostrom1yyx6wvgvr3deg70dvgsfwulgzurnz8kcwsna8a,cosmos1yyx6wvgvr3deg70dvgsfwulgzurnz8kcdr8we6,shock pipe begin detect economy finger great u...,bostrom1yyx6wvgvr3deg70dvgsfwulgzurnz8kcwsna8a...,0xcad3e647570e7f168aee49bbda543824c6dc6f47a06e...,{'pub_key': 'A1VGj0IPB2PAI0ZW25CScr//U/jJkxlDA...,eyJwdWJfa2V5IjoiQTFWR2owSVBCMlBBSTBaVzI1Q1Njci...,[eacf9cccca46c86697ff8c414e227c0289cd2596bbfe6...,[3fd71696db46abd392c10bd55ed9641d5819920f7cb5e...
4,30000,john16514110443,QmUCtjvizRdrdoF6e5VQRN5pgUB4D1wafwFzTsTjt2zPoY,0xa88f6812e62e6dbae520c8cc9bf2fbe68418f7cf,0xcd8696cdafd3388c6ca8a8062d3cf038dfca1129d559...,bostrom1yhrmy9l027gsa342wwxmua0pfel2c24egm037y,cosmos1yhrmy9l027gsa342wwxmua0pfel2c24etgmzqr,air video reflect suggest own skate spray mapl...,bostrom1yhrmy9l027gsa342wwxmua0pfel2c24egm037y...,0x58b5e9de07036ab4e5f2f4b185d3f0a684aca66e4678...,{'pub_key': 'AlWQq+xr7W2FW77LivQdpCQ/G8zmRucoE...,eyJwdWJfa2V5IjoiQWxXUXEreHI3VzJGVzc3TGl2UWRwQ1...,[e312e24f6ba1900bf48ad582159c77ff88a8048d00434...,[2389fa1c8f9edc9c945c480bd244203a383cee8908b52...


In [6]:
for i in range(ceil(NUMBER_OF_PARTICIPANTS/10_000)):
    claims_with_proofs_df.loc[i*10_000:(i+1)*10_000,:].to_csv(f'temp/{i}_{PARTICIPANTS_FILE_NAME}')

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [7]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph',
            from_address=WALLET_ADDRESS)
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph',
            from_address=WALLET_ADDRESS)
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom17p9rzwnnfxcjp32un9ug7yhhzgtkhvl9jfksztgw5uh69wac2pgs6qwudf
Avatar subgraph contract address: bostrom1unyuj8qnmygvzuex3dwmg9yzt9alhvyeat0uu0jedg2wj33efl5qpj0rgp
Proof subgraph contract address: bostrom1qwlgtx52gsdu7dtp0cekka5zehdl0uj3fhp9acg325fvgs8jdzksa7wx60


### Instantiate Passport Contract

In [8]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport',
            from_address=WALLET_ADDRESS)
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom18v47nqmhvejx3vc498pantg8vr435xa0rt6x0m6kzhp6yuqmcp8syatwkd


### Set executor in the Subgraph Contracts

In [9]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                                 contract_address=subgraph_contract_address,
                                 gas=600000,
                                 display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Treasury Contract

In [10]:
if INIT_TREASURY_CONTRACT:
    treasury_contract_address = \
        instantiate_contract(
            init_query=f'''{{"admins": ["{WALLET_ADDRESS}"], "mutable": true}}''',
            contract_code_id=TREASURY_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test treasury',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    treasury_contract_address = TREASURY_CONTRACT_ADDRESS
print(f'Treasury contract address: {treasury_contract_address}')

Treasury contract address: bostrom1xsmqvl8lqr2uwl50aetu0572rss9hrza5kddpfj9ky3jq80fv2tssevu8p


### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "treasury":"{treasury_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            contract_label='test gift',
            from_address=WALLET_ADDRESS,
            display_data=False)
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom14rse3e7rkc3qt7drmlulwlkrlzqvh7hv277zv05kyfuwl74udx5sugs6cp


### Initiate Class of Output Parsing

In [12]:
contract_utils = ContractUtils(ipfs_client=ipfs_client,
                               address_dict= {
                                   gift_contract_address: 'Gift Contract',
                                   passport_contract_address: 'Passport Contract',
                                   treasury_contract_address: 'Treasury Contract',
                                   WALLET_ADDRESS: 'Passport Owner Address',
                                   name_subgraph_contract_address: 'Name Subgraph Contract',
                                   avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                                   proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Set Allowance in Treasury contract

In [13]:
contract_utils.parse_contract_execution_json(
    contract_utils.send_coins(
        from_seed=WALLET_SEED,
        to_addresses=[treasury_contract_address],
        amounts=[INITIAL_BALANCE],
        gas=min(100_000 + 25_000, int(20e6)),
        display_data=DISPLAY_TX_EXECUTION))


Events

coin received
	receiver: Treasury Contract
	amount: 18000000000boot

coin spent
	spender: Passport Owner Address
	amount: 18000000000boot

message from bank bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0 /cosmos.bank.v1beta1.MsgMultiSend

transfer
	recipient: Treasury Contract
	amount: 18000000000boot
Gas used: 62,845
Tx hash: F1CFA2CD18343F891971826C2CD4A702C7C8CFD1B5E7B4FFF3B5B017A7F7A2F0


In [14]:
def set_allowance(amount: str = INITIAL_BALANCE, display_data: bool = False):
    return contract_utils.execute_contract(
        execute_msg={"increase_allowance": {"spender": gift_contract_address, "amount": {"amount": amount, "denom": 'boot'}}},
        contract_address=treasury_contract_address,
        mnemonic=WALLET_SEED,
        gas=600000,
        display_data=display_data)

contract_utils.parse_contract_execution_json(
    set_allowance(display_data=False))


Events

execute
	execute contract: Treasury Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Treasury Contract
	action: increase_allowance
	owner: Passport Owner Address
	spender: Gift Contract
	denomination: boot
	amount: 18,000,000,000
Gas used: 125,559
Tx hash: 9E75ECAB2CE7C193A95EE7802C722F0B058A6AC1152E9D36E5020FD50264D5C6


### Register Merkle Root

In [15]:
root_register_output = contract_utils.execute_contract(
    execute_msg={"register_merkle_root": {"merkle_root": root}},
    mnemonic=WALLET_SEED,
    contract_address=gift_contract_address)
contract_utils.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 41aaa12edbc734004495fd8995040aa2526f4cd0c40b58d7304e5f12258a767c
Gas used: 121,440
Tx hash: DF923569FD8ADB67237A9171F7C83DDDB54FCC0A463CC8FBD4230AC65B713E70


#### Get Merkle Root form the Gift Contract

In [16]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom14rse3e7rkc3qt7drmlulwlkrlzqvh7hv277zv05kyfuwl74udx5sugs6cp
{'data': {'merkle_root': '41aaa12edbc734004495fd8995040aa2526f4cd0c40b58d7304e5f12258a767c'}}


### Send coins to new addresses

In [17]:
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 750

bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in tqdm(bostrom_addresses_chunks):
    send_output = contract_utils.send_coins(
            from_seed=WALLET_SEED,
            to_addresses=bostrom_addresses_item,
            amounts=[1] * len(bostrom_addresses_item),
            gas=min(150_000 + 30_000 * len(bostrom_addresses_item), int(24e6)),
            display_data=DISPLAY_TX_EXECUTION)


  0%|          | 0/134 [00:00<?, ?it/s]

In [18]:
contract_utils.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1tafgegn8hset335n6kht23rweu7nu20cw3dnjp
	amount: 1boot
	receiver: bostrom1pc7saqwh689zn3cxm80kespxd6j4nhdtghyvcy
	amount: 1boot
	receiver: bostrom1axlxqvdq8pgrsk8vajjhuyup0tzlmsy3336mmd
	amount: 1boot
	receiver: bostrom18sudl5j6rrnc0mkl5z0l4tym4gkku4j280p3n4
	amount: 1boot
	receiver: bostrom1epctws6sr89ylvf0ugrcqx9nlfah8pzm8gdyp0
	amount: 1boot
	receiver: bostrom158j4whffw8dx03gr783hx5klgl08c6nqpe3v0r
	amount: 1boot
	receiver: bostrom1qccazrp3yucl8ppgkw6pgjks2a3m3yvjfnganl
	amount: 1boot
	receiver: bostrom12fdazv5zg4ssg2fvaaqdfp27c2zg6jef0n54n2
	amount: 1boot
	receiver: bostrom1xm7vkgavp22ueptqxhy6xf5535epg43lm52jn4
	amount: 1boot
	receiver: bostrom1dd2ugk0wx8rjzpgu3fxh40aw0xawkp60e0cwm3
	amount: 1boot
	receiver: bostrom1vajqcczskzr56rhumz40xfz9xkjqxrt9m2ajcl
	amount: 1boot
	receiver: bostrom15wcrqzhu0w7re5g87syzyn28jmp7pwtndz5xxa
	amount: 1boot
	receiver: bostrom1ylu3frh66ptt07szf47y50yxppuq4lkyf92mpn
	amount: 1boot
	receiver: bostrom16xshv0ey7

### Create Passports, Proof Addresses and Claim Gift

In [19]:
def execute_participation(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address}')
    if _output:
        return _output
    else:
        return _error

NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_participation = list(tqdm(pool.imap(execute_participation, tasks), total=len(tasks)))

Number of tasks: 50000
Number of threads: 100


  0%|          | 0/50000 [00:00<?, ?it/s]

In [20]:
with open('temp/contract_participation_execution_log_1.txt', 'r') as f:
    data = json.load(f)

In [21]:
contract_utils.parse_contract_execution_json(data['create'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	owner: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	token_id: 61
Gas used: 304,372
Tx hash: 479D19A1B8A50B1DE60F7F080544708DB4CFB3B69D3955A2BA24DD4854FEB647


In [22]:
contract_utils.parse_contract_execution_json(data['proof_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16514110440
	address: 0x3756069ce10d18b5d23db3295ec59f62909b4032
Gas used: 218,524
Tx hash: E346D4F7149646CD741DC5A23E5416AC26FDF110BB1ECBD48ECCF908C2F9C0C5


In [23]:
contract_utils.parse_contract_execution_json(data['proof_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Proof Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16514110440
	address: cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj
Gas used: 221,956
Tx hash: FC9B1C79E36B1975184F1ECDD5616DD9BFA7B84D1AAD77F58EC3FFE795ADA3FF


In [24]:
contract_utils.parse_contract_execution_json(data['claim_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	amount: 100000boot

coin spent
	spender: Treasury Contract
	amount: 100000boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	sender: Treasury Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj
	target: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	amount: 389,403
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 396,095
Tx hash: C8D1C15232ABDAC3B51FD0BE14572D9784C66530BA34C13CFF8A3B7C8FA22A91


### Release Gift

In [25]:
def execute_release(row_index: int) -> bool:
    _output, _error = execute_bash(bash_command=f'{PYTHON_PATH} create_passport_and_claim_job.py temp/{row_index//10_000}_{PARTICIPANTS_FILE_NAME} {row_index} {gift_contract_address} True')
    if _output:
        return _output
    else:
        return _error


NUMBER_OF_THREADS = 100

tasks = list(range(NUMBER_OF_ACTIVATED_PARTICIPANTS))
print(f'Number of tasks: {len(tasks)}')
print(f'Number of threads: {NUMBER_OF_THREADS}')


with Pool(processes=NUMBER_OF_THREADS) as pool:
    res_release = list(tqdm(pool.imap(execute_release, tasks), total=len(tasks)))

Number of tasks: 50000
Number of threads: 100


  0%|          | 0/50000 [00:00<?, ?it/s]

In [26]:
with open('temp/contract_release_execution_log_1.txt', 'r') as f:
    release_data = json.load(f)

In [27]:
contract_utils.parse_contract_execution_json(release_data['release_ethereum'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	amount: 28979boot

coin spent
	spender: Treasury Contract
	amount: 28979boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	sender: Treasury Contract
	amount: 28979boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	gift_address: 0x3756069ce10d18b5d23db3295ec59f62909b4032
	stage: 9
	amount: 28,979
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,339
Tx hash: 5886C678BD4A75924F53CC8444E2BE0C44FACB43D293B5494A890CE34FCC402E


In [28]:
contract_utils.parse_contract_execution_json(release_data['release_cosmos'], row=claims_with_proofs_df.loc[1,:])


Events

coin received
	receiver: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	amount: 28940boot

coin spent
	spender: Treasury Contract
	amount: 28940boot

execute
	execute contract: Gift Contract
	execute contract: Treasury Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	sender: Treasury Contract
	amount: 28940boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	gift_address: cosmos16uzvcpnw63wl8g5a5x5q9p2p0l7klgqwa4lquj
	stage: 9
	amount: 28,940
	_contract_address: Treasury Contract
	action: execute
	owner: Gift Contract
Gas used: 230,653
Tx hash: B876634CE093207DB1E447A53F6726167C49CC65ED7863AFD56A0C33BB6728FE


## Passport NFT testing
### Transfer Passport

In [29]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"tokens": {{"owner": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['tokens'][0]

In [30]:
for index, row in tqdm(claims_with_proofs_df[:4].iterrows()):
    if index == 1:
        transfer_passport_json = contract_utils.transfer_passport(
            row,
            token_id=get_passport_id(row['bostrom_address']),
            to_address=WALLET_ADDRESS,
            display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(
            transfer_passport_json,
            row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: transfer_nft
	sender: bostrom16uzvcpnw63wl8g5a5x5q9p2p0l7klgqw7xtnz4
	recipient: Passport Owner Address
	token_id: 61
Gas used: 306,332
Tx hash: 3F84A09B27A7A98A389EED1D231C575DEF8B64DED26FFC6B03852177363E36A6


### Burn

In [31]:
for index, row in tqdm(claims_with_proofs_df[2:6].iterrows()):
    if index == 2:
        burn_passport_json = contract_utils.burn_passport(row, token_id=get_passport_id(row['bostrom_address']), display_data=DISPLAY_TX_EXECUTION)
        contract_utils.parse_contract_execution_json(burn_passport_json, row=row)
    else:
        pass

0it [00:00, ?it/s]


Events

execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Name Subgraph Contract
	reply contract: Avatar Subgraph Contract

wasm
	_contract_address: Passport Contract
	action: burn
	sender: bostrom1mvuczu56l9c3rsj889ysykumpg4s9v3xgdq66l
	token_id: 76
Gas used: 275,513
Tx hash: F5248DF2DCF529A59B67C3533ECBBDA32F19AD473ECB97CE8AA4DB27BBF723A9


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
